<a href="https://colab.research.google.com/github/SaiBharath28/Task-Manager-With-Authentication/blob/main/Task_Manager_with_Authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install pyngrok --upgrade

In [12]:
!ngrok authtoken 2sDNMwlGR5NZ2O9Sh1LPpKHlrBn_7T51Haei38u6jgppqReRH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
!sudo apt update && sudo apt install ngrok

-----BEGIN PGP PUBLIC KEY BLOCK-----

mQINBGFGa/IBEAC0pgrns33QSiSipIKRnUCLeXX8d8WQjeqOqVEA6QlavAW54hKn
uHbbf151LFp0eGGaTp5ppJih2u9geiE8GxVACMuCHdlI0d35wAkucr+15qG+rfxm
qLkGGWARsRni3RJwg9R6/MwBgG6lB6mXOws2o/dnB8+wXnES5K9fBL2ZcSgtOQNn
Et0LWhZ3aJP79LMJn5yCuCI4lbKXMVL7LpLEM7gxOD6qjTkn6b34NliTeKwjXw9A
3qWj9UA++51q8lxDlGrklNlQMAUPXLXG22qEGhzmXAL88ldciED1iVL0tzofsVfj
hQPvt7i0xJd9/t42aK1ZtVUJdDjhoIDeZ4IRlT57K1cf3tUgGTD5xhnF7uZHJuqI
kOJc2+JwjLxRRnZE4PzaBzKKBYs/KM3F49Kv7IrlDl8ggwI9c7ReWJtgOOibtGgZ
AX1N+Hh6fNpWlgaf2VLTLU7m9gF1HDumXUC6gunQyzlNYJ4CsFm34E1r8lIMQAX3
SNWdeu119p0KhzCEqNlVxEqvldzoZ5oAqAnqIEVsvuQLblqCh6yDEuV5mTS6pE45
oMEq9jgBQ+mjqkzVpB7g8H7+gcFs/EBIv+V77fN8dqiw+7mKYOFSkvJTiyxnDgSK
0VKFsVRe8pfkrjzZUccPt0cWajoj2rJmD3JcVOcUlCcQnTH/QdKVLB6wewARAQAB
tDhuZ3JvayBhZ2VudCBhcHQgcmVwbyByZWxlYXNlIGJvdCA8cmVsZWFzZS1ib3RA
bmdyb2suY29tPokCTgQTAQgAOBYhBPAnH89xLPLjmQHxow5h07uq7jf+BQJhRmvy
AhsDBQsJCAcCBhUKCQgLAgQWAgMBAh4BAheAAAoJEA5h07uq7jf+0FIP/1RCUUkX
QlUzQDmZzusTfN2oeXN6OHk/4kq+qLlkbsdL11rwoRemGKUQu4Qk

In [14]:
import os

# Create templates directory if it doesn't exist
os.makedirs('templates', exist_ok=True)

# Base template
base_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Task Manager - {% block title %}{% endblock %}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <div class="container">
            <a class="navbar-brand" href="{{ url_for('index') }}">Task Manager</a>
            <div class="navbar-nav">
                {% if 'user_id' in session %}
                    <a class="nav-link" href="{{ url_for('index') }}">Tasks</a>
                    <a class="nav-link" href="{{ url_for('new_task') }}">New Task</a>
                    <a class="nav-link" href="{{ url_for('logout') }}">Logout</a>
                {% else %}
                    <a class="nav-link" href="{{ url_for('login') }}">Login</a>
                    <a class="nav-link" href="{{ url_for('register') }}">Register</a>
                {% endif %}
            </div>
        </div>
    </nav>

    <div class="container mt-4">
        {% with messages = get_flashed_messages() %}
            {% if messages %}
                {% for message in messages %}
                    <div class="alert alert-info">{{ message }}</div>
                {% endfor %}
            {% endif %}
        {% endwith %}

        {% block content %}{% endblock %}
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
'''

# Index template
index_template = '''
{% extends "base.html" %}
{% block title %}Home{% endblock %}
{% block content %}
    <h1>My Tasks</h1>
    <div class="row mt-4">
        {% for task in tasks %}
            <div class="col-md-4 mb-4">
                <div class="card">
                    <div class="card-body">
                        <h5 class="card-title">{{ task.title }}</h5>
                        <p class="card-text">{{ task.description }}</p>
                        <p class="card-text">
                            <small class="text-muted">
                                Due: {{ task.due_date.strftime('%Y-%m-%d') }}<br>
                                Status: {{ task.status }}
                            </small>
                        </p>
                        <a href="{{ url_for('edit_task', task_id=task.id) }}" class="btn btn-primary">Edit</a>
                        <a href="{{ url_for('delete_task', task_id=task.id) }}" class="btn btn-danger"
                           onclick="return confirm('Are you sure?')">Delete</a>
                    </div>
                </div>
            </div>
        {% endfor %}
    </div>
{% endblock %}
'''

# Login template
login_template = '''
{% extends "base.html" %}
{% block title %}Login{% endblock %}
{% block content %}
    <div class="row">
        <div class="col-md-6 offset-md-3">
            <h1>Login</h1>
            <form method="POST">
                <div class="mb-3">
                    <label for="username" class="form-label">Username</label>
                    <input type="text" class="form-control" id="username" name="username" required>
                </div>
                <div class="mb-3">
                    <label for="password" class="form-label">Password</label>
                    <input type="password" class="form-control" id="password" name="password" required>
                </div>
                <button type="submit" class="btn btn-primary">Login</button>
            </form>
        </div>
    </div>
{% endblock %}
'''

# Register template
register_template = '''
{% extends "base.html" %}
{% block title %}Register{% endblock %}
{% block content %}
    <div class="row">
        <div class="col-md-6 offset-md-3">
            <h1>Register</h1>
            <form method="POST">
                <div class="mb-3">
                    <label for="username" class="form-label">Username</label>
                    <input type="text" class="form-control" id="username" name="username" required>
                </div>
                <div class="mb-3">
                    <label for="email" class="form-label">Email</label>
                    <input type="email" class="form-control" id="email" name="email" required>
                </div>
                <div class="mb-3">
                    <label for="password" class="form-label">Password</label>
                    <input type="password" class="form-control" id="password" name="password" required>
                </div>
                <button type="submit" class="btn btn-primary">Register</button>
            </form>
        </div>
    </div>
{% endblock %}
'''

# New task template
new_task_template = '''
{% extends "base.html" %}
{% block title %}New Task{% endblock %}
{% block content %}
    <div class="row">
        <div class="col-md-6 offset-md-3">
            <h1>New Task</h1>
            <form method="POST">
                <div class="mb-3">
                    <label for="title" class="form-label">Title</label>
                    <input type="text" class="form-control" id="title" name="title" required>
                </div>
                <div class="mb-3">
                    <label for="description" class="form-label">Description</label>
                    <textarea class="form-control" id="description" name="description" rows="3"></textarea>
                </div>
                <div class="mb-3">
                    <label for="due_date" class="form-label">Due Date</label>
                    <input type="date" class="form-control" id="due_date" name="due_date" required>
                </div>
                <button type="submit" class="btn btn-primary">Create Task</button>
            </form>
        </div>
    </div>
{% endblock %}
'''

# Edit task template
edit_task_template = '''
{% extends "base.html" %}
{% block title %}Edit Task{% endblock %}
{% block content %}
    <div class="row">
        <div class="col-md-6 offset-md-3">
            <h1>Edit Task</h1>
            <form method="POST">
                <div class="mb-3">
                    <label for="title" class="form-label">Title</label>
                    <input type="text" class="form-control" id="title" name="title" value="{{ task.title }}" required>
                </div>
                <div class="mb-3">
                    <label for="description" class="form-label">Description</label>
                    <textarea class="form-control" id="description" name="description" rows="3">{{ task.description }}</textarea>
                </div>
                <div class="mb-3">
                    <label for="due_date" class="form-label">Due Date</label>
                    <input type="date" class="form-control" id="due_date" name="due_date"
                           value="{{ task.due_date.strftime('%Y-%m-%d') }}" required>
                </div>
                <div class="mb-3">
                    <label for="status" class="form-label">Status</label>
                    <select class="form-control" id="status" name="status">
                        <option value="pending" {% if task.status == 'pending' %}selected{% endif %}>Pending</option>
                        <option value="in_progress" {% if task.status == 'in_progress' %}selected{% endif %}>In Progress</option>
                        <option value="completed" {% if task.status == 'completed' %}selected{% endif %}>Completed</option>
                    </select>
                </div>
                <button type="submit" class="btn btn-primary">Update Task</button>
            </form>
        </div>
    </div>
{% endblock %}
'''

# Write templates to files
templates = {
    'base.html': base_template,
    'index.html': index_template,
    'login.html': login_template,
    'register.html': register_template,
    'new_task.html': new_task_template,
    'edit_task.html': edit_task_template
}

for filename, content in templates.items():
    with open(os.path.join('templates', filename), 'w') as f:
        f.write(content.strip())

print("Templates created successfully!")

# Verify the files were created
print("\nCreated template files:")
for filename in os.listdir('templates'):
    print(f"- templates/{filename}")

Templates created successfully!

Created template files:
- templates/base.html
- templates/index.html
- templates/register.html
- templates/login.html
- templates/new_task.html
- templates/edit_task.html


In [15]:
!pip install flask flask-sqlalchemy werkzeug pyngrok

In [16]:
!mkdir -p templates

In [17]:
import secrets
secret_key = secrets.token_hex(16)
print(f"Your Generated Secret Key: {secret_key}")

Your Generated Secret Key: 7fbad2c2b5113d6e20b3b0edaf19bb5e


In [18]:
# app.py
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
from datetime import datetime
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = '0a5fa17aac2e6dcac20e4cd5ebba1563'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///tasks.db'
db = SQLAlchemy(app)

# Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password_hash = db.Column(db.String(120), nullable=False)
    tasks = db.relationship('Task', backref='owner', lazy=True)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text)
    due_date = db.Column(db.DateTime)
    status = db.Column(db.String(20), default='pending')
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

# Routes
@app.route('/')
def index():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    tasks = Task.query.filter_by(user_id=session['user_id']).all()
    return render_template('index.html', tasks=tasks)

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']

        if User.query.filter_by(username=username).first():
            flash('Username already exists')
            return redirect(url_for('register'))

        user = User(username=username, email=email)
        user.set_password(password)
        db.session.add(user)
        db.session.commit()

        flash('Registration successful')
        return redirect(url_for('login'))

    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()

        if user and user.check_password(password):
            session['user_id'] = user.id
            flash('Logged in successfully')
            return redirect(url_for('index'))

        flash('Invalid username or password')
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('user_id', None)
    return redirect(url_for('login'))

@app.route('/task/new', methods=['GET', 'POST'])
def new_task():
    if 'user_id' not in session:
        return redirect(url_for('login'))

    if request.method == 'POST':
        title = request.form['title']
        description = request.form['description']
        due_date = datetime.strptime(request.form['due_date'], '%Y-%m-%d')

        task = Task(
            title=title,
            description=description,
            due_date=due_date,
            user_id=session['user_id']
        )
        db.session.add(task)
        db.session.commit()

        flash('Task created successfully')
        return redirect(url_for('index'))

    return render_template('new_task.html')

@app.route('/task/<int:task_id>/edit', methods=['GET', 'POST'])
def edit_task(task_id):
    if 'user_id' not in session:
        return redirect(url_for('login'))

    task = Task.query.get_or_404(task_id)
    if task.user_id != session['user_id']:
        flash('Unauthorized access')
        return redirect(url_for('index'))

    if request.method == 'POST':
        task.title = request.form['title']
        task.description = request.form['description']
        task.due_date = datetime.strptime(request.form['due_date'], '%Y-%m-%d')
        task.status = request.form['status']
        db.session.commit()

        flash('Task updated successfully')
        return redirect(url_for('index'))

    return render_template('edit_task.html', task=task)

@app.route('/task/<int:task_id>/delete')
def delete_task(task_id):
    if 'user_id' not in session:
        return redirect(url_for('login'))

    task = Task.query.get_or_404(task_id)
    if task.user_id != session['user_id']:
        flash('Unauthorized access')
        return redirect(url_for('index'))

    db.session.delete(task)
    db.session.commit()
    flash('Task deleted successfully')
    return redirect(url_for('index'))

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from pyngrok import ngrok

# Setup ngrok
public_url = ngrok.connect(5000)
print(f' * Public URL: {public_url}')

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run()

 * Public URL: NgrokTunnel: "https://b558-34-16-179-214.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:00] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:00] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:10] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:17] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:18] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:24] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:29] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Jan/2025 18:27:42] "GET / HTTP/1.1" 200 -
INFO